In [2]:
import os
import json
import sqlite3
from openai import OpenAI
import logging
from tqdm import tqdm
from dotenv import load_dotenv
from openai import RateLimitError
import backoff
import yaml
from LLMDataProcessing import create_db_tables, process_all_prompts, fetch_prompts_from_db, save_results_to_db

In [3]:
# Load environment variables from .env file
load_dotenv()

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Initialize OpenAI client
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
@backoff.on_exception(backoff.expo, RateLimitError)

def main():
    # Load environment variables
    load_dotenv()

    # Set up logging
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

    # Initialize OpenAI client
    client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

    # Load configuration from YAML file
    with open('config.yaml', 'r') as config_file:
        config = yaml.safe_load(config_file)

    # Database path
    db_path = config['db_path']
    
    
    # Ensure database file exists
    if not os.path.exists(db_path):
        logging.error(f"Database file not found: {db_path}")
        return

    try:
        # Connect to the database
        with sqlite3.connect(db_path) as conn:
            # Create necessary tables
            create_db_tables(conn)
            
            # Process all prompts from the newspaper_data table
            process_all_prompts(conn, client)
            
            logging.info("Processing completed successfully.")
    
    except sqlite3.Error as e:
        logging.error(f"SQLite error: {e}")
    except Exception as e:
        logging.error(f"An unexpected error occurred: {e}")
    finally:
        logging.info("Script execution completed.")

if __name__ == "__main__":
    main()

Processing prompts:   0%|          | 0/838 [00:00<?, ?it/s]2024-07-25 15:34:55,768 - INFO - Skipping already processed custom_id: dark-3693573-1-4-0
2024-07-25 15:34:55,768 - INFO - Skipping already processed custom_id: dark-3693573-1-4-5
2024-07-25 15:34:55,769 - INFO - Skipping already processed custom_id: dark-3693573-1-4-6
2024-07-25 15:34:55,770 - INFO - Skipping already processed custom_id: dark-3693573-1-4-8
2024-07-25 15:34:55,770 - INFO - Skipping already processed custom_id: dark-3693573-1-4-9
2024-07-25 15:34:55,770 - INFO - Skipping already processed custom_id: dark-3693573-1-4-10
2024-07-25 15:34:55,771 - INFO - Skipping already processed custom_id: dark-3693566-1-4-11
2024-07-25 15:34:55,771 - INFO - Skipping already processed custom_id: dark-3693566-1-4-12
2024-07-25 15:34:55,772 - INFO - Skipping already processed custom_id: dark-3693566-1-4-13
2024-07-25 15:34:55,772 - INFO - Skipping already processed custom_id: dark-3693566-1-4-15
2024-07-25 15:34:55,772 - INFO - Ski